# Session 5: Linear Regression

## Session Introduction & Objectives

Welcome to Session 5! Today, we'll dive into one of the most fundamental and widely used algorithms in machine learning: **Linear Regression**. We'll build upon our data analysis and visualization skills to create predictive models.

### Learning Objectives
By the end of this session, you will be able to:
- Understand the theory behind Linear Regression.
- Implement a Linear Regression model using scikit-learn.
- Prepare data for a regression model (train-test split, scaling).
- Evaluate the performance of a Linear Regression model.
- Interpret the results of the model.

### What We'll Cover
1. **What is Linear Regression?**
2. **Loading a Real-World Dataset**
3. **Exploratory Data Analysis (EDA)**
4. **Data Preprocessing**
5. **Training the Linear Regression Model**
6. **Model Evaluation**
7. **Visualizing Results**
8. **Hands-on Practice**

## 1. What is Linear Regression?

Linear Regression is a statistical method used to model the relationship between a dependent variable and one or more independent variables. The goal is to find the best-fitting straight line (or hyperplane) that describes the data.

### Key Concepts

- **Dependent Variable (Target, y):** The variable we are trying to predict.
- **Independent Variable(s) (Features, X):** The variables we use to make the prediction.
- **Simple Linear Regression:** One independent variable. The formula is: `y = β₀ + β₁x + ε`
- **Multiple Linear Regression:** Two or more independent variables. The formula is: `y = β₀ + β₁x₁ + β₂x₂ + ... + βₙxₙ + ε`

Where:
- `β₀` is the intercept (the value of y when all x's are 0).
- `β₁...βₙ` are the coefficients (the change in y for a one-unit change in x).
- `ε` is the error term.

### Assumptions of Linear Regression
For a linear regression model to be accurate and reliable, several key assumptions about the data must hold true.

1.  **Linearity:** The relationship between the independent variables (X) and the dependent variable (y) must be linear. You can check this by creating scatter plots of each feature against the target. If the pattern is not linear, you might need to transform the data (e.g., log transformation) or use a different, non-linear model.

2.  **Independence:** The observations in the dataset must be independent of each other. This means that the residual (error) for one observation is not correlated with the residual of another. This assumption is often violated in time-series data, where one observation depends on the previous one.

3.  **Homoscedasticity (Constant Variance):** The variance of the residuals should be constant across all levels of the independent variables. In other words, the spread of the errors should be consistent. If the variance increases as the predicted value increases (a funnel shape in the residual plot), this is called heteroscedasticity and can make the model's predictions less reliable.

4.  **Normality of Residuals:** The residuals of the model should be normally distributed. This doesn't mean the features or target must be normally distributed, but their errors should be. You can check this by plotting a histogram of the residuals. If this assumption is violated, the p-values and confidence intervals for the coefficients can be unreliable.

5.  **No Multicollinearity:** The independent variables should not be highly correlated with each other. If there is high correlation (multicollinearity), it becomes difficult for the model to determine the individual effect of each feature on the target variable. You can check for this using a correlation matrix or by calculating the Variance Inflation Factor (VIF).

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import statsmodels.api as sm

# Set up matplotlib for inline plotting
%matplotlib inline

# Set default figure size
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_style('whitegrid')

print("Libraries imported successfully!")

## 2. Loading a Real-World Dataset

We will use the **California Housing dataset**, which is available in scikit-learn. This dataset contains information about housing districts in California from the 1990 census. Our goal is to predict the median house value for a district.

In [ ]:
# Load the dataset
housing = fetch_california_housing()

# Create a pandas DataFrame
df = pd.DataFrame(housing.data, columns=housing.feature_names)
df['MedHouseVal'] = housing.target

print("Dataset loaded successfully!")
print("Shape of the dataset:", df.shape)
df.head()

## 3. Exploratory Data Analysis (EDA)

Let's explore the dataset to understand its structure and the relationships between variables.

In [ ]:
# Get basic information about the dataset
print("Dataset Info:")
df.info()

In [ ]:
# Get summary statistics
print("\nSummary Statistics:")
df.describe()

In [ ]:
# Check for missing values
print("\nMissing Values:")
print(df.isnull().sum())

### Visualizing the Data
Let's create some visualizations to understand the data better.

In [ ]:
# Distribution of the target variable (Median House Value)
plt.figure(figsize=(10, 6))
sns.histplot(df['MedHouseVal'], bins=30, kde=True)
plt.title('Distribution of Median House Value', fontsize=16, fontweight='bold')
plt.xlabel('Median House Value ($100,000s)')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Correlation Heatmap
plt.figure(figsize=(12, 10))
correlation_matrix = df.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix of Housing Features', fontsize=16, fontweight='bold')
plt.show()

From the heatmap, we can see that `MedInc` (Median Income) has the strongest positive correlation with our target `MedHouseVal`. Let's visualize this relationship with a scatter plot.

In [ ]:
# Scatter plot of Median Income vs Median House Value
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='MedInc', y='MedHouseVal', alpha=0.4)
plt.title('Median Income vs. Median House Value', fontsize=16, fontweight='bold')
plt.xlabel('Median Income ($10,000s)')
plt.ylabel('Median House Value ($100,000s)')
plt.show()

## 4. Data Preprocessing

Before training our model, we need to prepare the data. This involves:
1.  **Separating features (X) and target (y).**
2.  **Splitting the data** into training and testing sets.
3.  **Scaling the features** to ensure they are on a similar scale.

In [ ]:
# 1. Separate features and target
X = df.drop('MedHouseVal', axis=1)
y = df['MedHouseVal']

# 2. Split data into training and testing sets
# We'll use 80% for training and 20% for testing.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set shape: {X_train.shape}")
print(f"Testing set shape: {X_test.shape}")

# 3. Scale the features
# It's important to fit the scaler on the training data ONLY, then transform both train and test data.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Using `statsmodels` for more detailed analysis
While `scikit-learn` is great for predictive modeling, the `statsmodels` library provides a more detailed statistical summary of the regression model, which is excellent for inferential analysis. Let's train a model with it as well.


In [ ]:
# statsmodels requires us to add a constant to our features (for the intercept)
X_train_sm = sm.add_constant(X_train_scaled)
X_test_sm = sm.add_constant(X_test_scaled)

# Create and train the model
sm_model = sm.OLS(y_train, X_train_sm).fit()

# Print the summary
print(sm_model.summary())

## 5. Training the Linear Regression Model

Now we are ready to train our model!

In [ ]:
# Create and train the model
lin_reg = LinearRegression()
lin_reg.fit(X_train_scaled, y_train)

print("Linear Regression model trained successfully!")

### Interpreting the Model
Let's look at the intercept and coefficients of our model.

In [ ]:
# Print the intercept
print(f"Intercept (β₀): {lin_reg.intercept_:.2f}")

# Create a DataFrame for the coefficients
coeff_df = pd.DataFrame(lin_reg.coef_, X.columns, columns=['Coefficient'])
print("\nCoefficients (β₁...βₙ):")
print(coeff_df.sort_values('Coefficient', ascending=False))

**Interpretation of Coefficients:**
The coefficients tell us how a one-unit increase in a scaled feature affects the median house value, holding all other features constant. For example, a one-unit increase in scaled `MedInc` is associated with an increase of approximately $83,000 in the median house value.

## 6. Model Evaluation

Now that we have a trained model, let's evaluate its performance on the unseen test data.

In [ ]:
# Make predictions on the test set
y_pred = lin_reg.predict(X_test_scaled)

We will use the following metrics to evaluate our model:
-   **Mean Absolute Error (MAE):** The average of the absolute differences between predicted and actual values.
-   **Mean Squared Error (MSE):** The average of the squared differences. Punishes larger errors more.
-   **Root Mean Squared Error (RMSE):** The square root of MSE. It's in the same units as the target variable.
-   **R-squared (R²):** The proportion of the variance in the dependent variable that is predictable from the independent variables. Ranges from 0 to 1.

In [ ]:
# Calculate evaluation metrics
mae = metrics.mean_absolute_error(y_test, y_pred)
mse = metrics.mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = metrics.r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-squared (R²): {r2:.2f}")

**Evaluation Interpretation:**
-   **RMSE:** Our model's predictions are, on average, off by about $72,000.
-   **R²:** Our model explains about 58% of the variability in median house values. This is a decent start but suggests there's room for improvement, perhaps with more complex models.

## 7. Visualizing Results

Visualizing the model's performance can provide more insight.

In [ ]:
# Scatter plot of Actual vs. Predicted values
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.4)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2) # Perfect prediction line
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title("Actual vs. Predicted Median House Values", fontsize=16, fontweight='bold')
plt.show()

Ideally, the points should fall along the red dashed line. Our model follows the trend, but there's significant scatter, confirming our R² value.

In [ ]:
# Residuals Plot
# Residuals are the difference between the actual and predicted values.
residuals = y_test - y_pred

plt.figure(figsize=(10, 6))
sns.histplot(residuals, bins=30, kde=True)
plt.title("Distribution of Residuals", fontsize=16, fontweight='bold')
plt.xlabel("Residuals (Actual - Predicted)")
plt.show()

The residuals should be roughly normally distributed and centered around zero, which seems to be the case here. This indicates that our model's errors are random and not systematically biased.

## Summary & Key Takeaways

In this session, we successfully built a Linear Regression model to predict California housing prices.

✅ We understood the theory and assumptions of Linear Regression.
✅ We loaded and explored a real-world dataset.
✅ We performed essential preprocessing steps like train-test split and feature scaling.
✅ We trained a model and interpreted its coefficients.
✅ We evaluated the model's performance using key regression metrics (MAE, MSE, RMSE, R²).
✅ We visualized the model's predictions and residuals.

### What's Next?
- **Improving the Model:** We could try polynomial regression to capture non-linear relationships or use more advanced models like Ridge or Lasso regression.
- **Feature Engineering:** Creating new features from existing ones might improve performance.

## Practice Exercises

1.  **Change the `random_state`** in `train_test_split`. Does the model's performance change? Why?
2.  **Train the model without feature scaling.** How do the evaluation metrics compare? Why is scaling important?
3.  **Select only the top 3 correlated features** from the EDA step and train a new model. How does it perform compared to the model with all features?
4.  **Interpret the coefficient for `AveRooms`.** What does it tell you about the relationship between the average number of rooms and the house value?

In [ ]:
# Space for your practice exercises

# Exercise 3: Train on top 3 features
# Your code here

## Additional Examples with Different Datasets

To solidify our understanding, let's apply linear regression to a few more real-world datasets.

### Example 1: Predicting Medical Insurance Costs

In this example, we'll use a dataset of medical insurance costs to predict a person's medical expenses based on their age, sex, BMI, number of children, and whether they are a smoker.

**Dataset Source:** [Medical Cost Personal Datasets on Kaggle](https://www.kaggle.com/datasets/mirichoi0218/insurance) (originally from GitHub)

In [ ]:
# Load the dataset from a public GitHub repository
url = 'https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv'
insurance_df = pd.read_csv(url)

# --- 1. Preprocessing ---
# Convert categorical variables to numerical
insurance_df_processed = pd.get_dummies(insurance_df, columns=['sex', 'smoker', 'region'], drop_first=True)

# Define features (X) and target (y)
X_ins = insurance_df_processed.drop('charges', axis=1)
y_ins = insurance_df_processed['charges']

# Split data
X_train_ins, X_test_ins, y_train_ins, y_test_ins = train_test_split(X_ins, y_ins, test_size=0.2, random_state=42)

# Scale features
scaler_ins = StandardScaler()
X_train_ins_scaled = scaler_ins.fit_transform(X_train_ins)
X_test_ins_scaled = scaler_ins.transform(X_test_ins)

# --- 2. Model Training ---
ins_model = LinearRegression()
ins_model.fit(X_train_ins_scaled, y_train_ins)

# --- 3. Evaluation ---
y_pred_ins = ins_model.predict(X_test_ins_scaled)
r2_ins = metrics.r2_score(y_test_ins, y_pred_ins)
rmse_ins = np.sqrt(metrics.mean_squared_error(y_test_ins, y_pred_ins))

print("--- Medical Insurance Cost Prediction ---")
print(f"Model R-squared: {r2_ins:.2f}")
print(f"Model RMSE: ${rmse_ins:,.2f}")

# --- 4. Visualization ---
plt.figure(figsize=(10, 6))
plt.scatter(y_test_ins, y_pred_ins, alpha=0.5)
plt.plot([y_test_ins.min(), y_test_ins.max()], [y_test_ins.min(), y_test_ins.max()], 'r--', lw=2)
plt.title('Actual vs. Predicted Insurance Charges', fontsize=16, fontweight='bold')
plt.xlabel('Actual Charges ($)')
plt.ylabel('Predicted Charges ($)')
plt.show()

# --- 5. Coefficient Analysis ---
coeffs = pd.DataFrame(ins_model.coef_, index=X_ins.columns, columns=['Coefficient'])
print("\nCoefficients:")
print(coeffs.sort_values('Coefficient', ascending=False))

### Example 2: Predicting Fish Weight

Here, we'll predict the weight of a fish based on its physical measurements (length, height, width) and species. This is a classic regression problem.

**Dataset Source:** [Fish Market on Kaggle](https://www.kaggle.com/datasets/aungpyaeap/fish-market) (originally from GitHub)

In [ ]:
# Load the dataset
url = 'https://raw.githubusercontent.com/marcopeix/datasciencewithmarco/master/data/Fish.csv'
fish_df = pd.read_csv(url)

# --- 1. Preprocessing ---
# Convert categorical species to numerical
fish_df_processed = pd.get_dummies(fish_df, columns=['Species'], drop_first=True)

# Define features (X) and target (y)
X_fish = fish_df_processed.drop('Weight', axis=1)
y_fish = fish_df_processed['Weight']

# Split data
X_train_fish, X_test_fish, y_train_fish, y_test_fish = train_test_split(X_fish, y_fish, test_size=0.2, random_state=42)

# Scale features
scaler_fish = StandardScaler()
X_train_fish_scaled = scaler_fish.fit_transform(X_train_fish)
X_test_fish_scaled = scaler_fish.transform(X_test_fish)

# --- 2. Model Training ---
fish_model = LinearRegression()
fish_model.fit(X_train_fish_scaled, y_train_fish)

# --- 3. Evaluation ---
y_pred_fish = fish_model.predict(X_test_fish_scaled)
r2_fish = metrics.r2_score(y_test_fish, y_pred_fish)
rmse_fish = np.sqrt(metrics.mean_squared_error(y_test_fish, y_pred_fish))

print("--- Fish Weight Prediction ---")
print(f"Model R-squared: {r2_fish:.2f}")
print(f"Model RMSE: {rmse_fish:.2f} grams")

# --- 4. Visualization ---
plt.figure(figsize=(10, 6))
plt.scatter(y_test_fish, y_pred_fish, alpha=0.6)
plt.plot([y_test_fish.min(), y_test_fish.max()], [y_test_fish.min(), y_test_fish.max()], 'r--', lw=2)
plt.title('Actual vs. Predicted Fish Weight', fontsize=16, fontweight='bold')
plt.xlabel('Actual Weight (g)')
plt.ylabel('Predicted Weight (g)')
plt.show()

### Example 3: Predicting Car Prices

In this final example, we'll predict the price of a car based on various features like its make, model, year, engine size, and horsepower.

**Dataset Source:** [Car Price Prediction on Kaggle](https://www.kaggle.com/datasets/hellbuoy/car-price-prediction) (originally from GitHub)

In [ ]:
# Load the dataset
url = 'https://raw.githubusercontent.com/hellbuoy/car-price-prediction/master/CarPrice_Assignment.csv'
car_df = pd.read_csv(url)

# --- 1. Preprocessing ---
# Select only numeric columns for simplicity
car_numeric = car_df.select_dtypes(include=np.number)
car_numeric = car_numeric.drop('car_ID', axis=1) # Drop irrelevant ID
car_numeric.dropna(inplace=True) # Drop rows with missing values

# Define features (X) and target (y)
X_car = car_numeric.drop('price', axis=1)
y_car = car_numeric['price']

# Split data
X_train_car, X_test_car, y_train_car, y_test_car = train_test_split(X_car, y_car, test_size=0.2, random_state=42)

# Scale features
scaler_car = StandardScaler()
X_train_car_scaled = scaler_car.fit_transform(X_train_car)
X_test_car_scaled = scaler_car.transform(X_test_car)

# --- 2. Model Training ---
car_model = LinearRegression()
car_model.fit(X_train_car_scaled, y_train_car)

# --- 3. Evaluation ---
y_pred_car = car_model.predict(X_test_car_scaled)
r2_car = metrics.r2_score(y_test_car, y_pred_car)
rmse_car = np.sqrt(metrics.mean_squared_error(y_test_car, y_pred_car))

print("--- Car Price Prediction ---")
print(f"Model R-squared: {r2_car:.2f}")
print(f"Model RMSE: ${rmse_car:,.2f}")

# --- 4. Visualization ---
plt.figure(figsize=(10, 6))
plt.scatter(y_test_car, y_pred_car, alpha=0.6)
plt.plot([y_test_car.min(), y_test_car.max()], [y_test_car.min(), y_test_car.max()], 'r--', lw=2)
plt.title('Actual vs. Predicted Car Prices', fontsize=16, fontweight='bold')
plt.xlabel('Actual Price ($)')
plt.ylabel('Predicted Price ($)')
plt.show()